In [912]:
import numpy as np
import pandas as pd
from scipy.stats import truncnorm
import matplotlib.pyplot as plt
import random
import tqdm

In [913]:
Distribution_Center()

In [914]:
template = {'food': 0, 'water':0, 'medical': 0}

In [915]:
class Distribution_Center(object):
    def __init__(self, dc_template=template, limit = 1000, nstores = 10, s_template=template):
        self.stores = []
        self.limit = limit
        for k, v in dc_template.items():
            setattr(self, k, v)
            
        for idx, store in enumerate(range(nstores)):
                self.stores.append(Store(s_template))
            
    def change_attr(self, attr, rate):
        val = getattr(self, attr)
        newval = val + rate
        if newval < 0:
            newval = 0
        if newval > self.limit:
            newval = self.limit
        setattr(self, attr, newval)
        return(val-newval)
    
    def reorder(self):
        for store in self.stores:
            for supply in store.template:
                currentval = getattr(store, supply)
                if currentval < store.re:
                    nextval = currentval + self.change_attr(supply, currentval - store.limit)
                    if nextval == 0:
                        print('not enough supply of {}'.format(supply))
                    setattr(store, supply, nextval)
                
        
    
            
    

In [916]:
class Store(Distribution_Center):
    def __init__(self, s_template=template, limit=100, re=20):
        self.template = s_template
        self.limit=limit
        self.re = re
        for k, v in s_template.items():
            setattr(self, k, v)
        super(Distribution_Center, self).__init__()
        
    

In [917]:
aldi = Distribution_Center(dc_template={'food': 1000, 'water':1000, 'medical': 1000})
def run_experiment(t=100):
    for i in range(t):
        
        print('aldi has {} food {} water {} medical'.format(aldi.food, aldi.water, aldi.medical))

        print('\n')

        for idx, stores in enumerate(aldi.stores):
            print('store {} has {} food {} water {} medical'.format(idx, stores.food, stores.water, stores.medical))
        
        print('\n')
        
        if aldi.change_attr('food', 30) == 0:
            print('surplus of food')
        if aldi.change_attr('water', 30) == 0:
            print('surplus of water')
        if aldi.change_attr('medical', 30) == 0:
            print('surplus of medical')
        
        aldi.reorder()
        
        
        

In [918]:
run_experiment()

aldi has 1000 food 1000 water 1000 medical


store 0 has 0 food 0 water 0 medical
store 1 has 0 food 0 water 0 medical
store 2 has 0 food 0 water 0 medical
store 3 has 0 food 0 water 0 medical
store 4 has 0 food 0 water 0 medical
store 5 has 0 food 0 water 0 medical
store 6 has 0 food 0 water 0 medical
store 7 has 0 food 0 water 0 medical
store 8 has 0 food 0 water 0 medical
store 9 has 0 food 0 water 0 medical


surplus of food
surplus of water
surplus of medical
aldi has 0 food 0 water 0 medical


store 0 has 100 food 100 water 100 medical
store 1 has 100 food 100 water 100 medical
store 2 has 100 food 100 water 100 medical
store 3 has 100 food 100 water 100 medical
store 4 has 100 food 100 water 100 medical
store 5 has 100 food 100 water 100 medical
store 6 has 100 food 100 water 100 medical
store 7 has 100 food 100 water 100 medical
store 8 has 100 food 100 water 100 medical
store 9 has 100 food 100 water 100 medical


aldi has 30 food 30 water 30 medical


store 0 has 100 food 100

In [893]:
print('aldi has {} food {} water {} medical'.format(aldi.food, aldi.water, aldi.medical))

print('\n')

for idx, stores in enumerate(aldi.stores):
    print('store {} has {} food {} water {} medical'.format(idx, stores.food, stores.water, stores.medical))

aldi has 0 food 0 water 0 medical


store 0 has 100 food 100 water 100 medical
store 1 has 70 food 70 water 70 medical
store 2 has 100 food 100 water 100 medical
store 3 has 70 food 70 water 70 medical
store 4 has 100 food 100 water 100 medical
store 5 has 70 food 70 water 70 medical
store 6 has 0 food 0 water 0 medical
store 7 has 70 food 70 water 70 medical
store 8 has 0 food 0 water 0 medical
store 9 has 70 food 70 water 70 medical
